# Neural networks

*Megan Courtman <megan.courtman@plymouth.ac.uk>*

In this notebook, we will look at using Python libraries to apply neural networks to a medical imaging problem. 

## Definitions

### Artificial intelligence
Artificial intelligence (AI) is a broad term, coined in 1955, which generally describes agents that receive information from the environment and perform actions.

### Machine learning
Machine learning is a subset of AI, and describes algorithms that improve themselves without explicit instruction through exposure to data. There are many types of machine learning, many of which require manual feature selection.

### Deep learning
Deep learning is a subset of machine learning, in which layered neural networks are used to build representations of complicated concepts out of simpler concepts. In neural networks, the data passes from node to node in a structure that loosely resembles neurons in the brain. A distinctive aspect of deep learning is that it does not require manual feature selection or engineering, and can learn more abstract representations.

<div>
<img src="neural_network.png" width="400"/>
</div>

#### Convolutional neural networks
A convolutional neural network (CNN) is a specific type of neural network that is used in image analysis. At each convolutional stage, a filter passes over the image to detect the presence of patterns. At the beginning of the network it picks up simple patterns (like straight lines, curves or corners). Later in the network these patterns will be combined to detect more complicated patterns. See image below for an example of CNN architecture, and the gif below for a visualisation of how the filter moves across the image.

<div>
<img src="cnn.png" width="750"/>
</div>

<div>
<img src="cnn.gif" width="200"/>
</div>

## AI and deep learning in radiology
Deep learning has been successfully applied to computer vision tasks such as object classification, localisation and detection. Many attempts have been made to translate these successes into radiological applications. There are now thousands of publications applying computer vision techniques to medical imaging.

### Advantages
The integration of AI into the radiological assessment workflow to provide radiologists with pre-screened images and identified features may have great benefits to offer, including enhanced efficiency and efficacy in diagnostics and the reduction of observational oversights and omissions. AI has the potential to detect patterns which are invisible to the human eye. It also has the potential to make healthcare more equitable, by being data-driven and (theoretically) not subject to human biases.

### Disadvantages
There have been many cases of AI being misused or poorly understood in healthcare research. An example which well illustrates many of the common pitfalls is the application of machine learning to detecting COVID-19 in chest radiographs and CT scans. Over 2000 papers were published on this subject between January and October 2020, but a [review](https://www.nature.com/articles/s42256-021-00307-0) found that none of the models in the 62 included studies were of  clinical use due to methodological flaws or underlying biases. It is very important that clinicians and computer scientists work together to foresee and mitigate such pitfalls. 

## Python libraries for deep learning

In this notebook we will be using [Keras](https://keras.io/), a neural network library for Python. This sits on top of [TensorFlow](https://www.tensorflow.org/), a machine learning library developed by Google. These are open-source libraries, which means that the source code is freely available for use and modification.

We will also be using other open-source Python libraries which will help us process and analyse the data. These include [Matplotlib](https://matplotlib.org/) for plotting images, [OpenCV](https://docs.opencv.org/4.x/) and [imutils](https://github.com/PyImageSearch/imutils) for image processing, [NumPy](https://numpy.org/) for working with arrays, [os](https://docs.python.org/3/library/os.html) for interacting with the operating system, and [Scikit-learn](https://scikit-learn.org/stable/index.html) for machine learning functionality.

## Example: brain tumour detection

For a quick example of using deep learning in Python for medical imaging, we will look at detecting brain tumours in MRI scans. We will use [a publicly available dataset hosted on Kaggle](https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection).